In [1]:
# Libraries
import pandas as pd
from collections import Counter
from functools import reduce
import numpy as np
from ast import literal_eval  # Import literal_eval to safely evaluate string representation of lists


# 2. Grouping Users together!
Now, we will deal with clustering algorithms that will provide groups of Netflix users that are similar among them.

To solve this task, you must accomplish the following stages:

## 2.1 Getting your data + feature engineering
Access to the data found in this dataset

Sometimes, the features (variables, fields) are not given in a dataset but can be created from it; this is known as feature engineering. For example, the original dataset has several clicks done by the same user, so grouping data by user_id will allow you to create new features for each user:

a) Favorite genre (i.e., the genre on which the user spent the most time)

b) Average click duration

c) Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)

d) Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?

e) Average time spent a day by the user (considering only the days he logs in)

In [2]:
# Load data
data = pd.read_csv("vodclickstream_uk_movies_03.csv")
not_clean = data

In [3]:
not_clean.head()

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


In [4]:
# Cleaning data and adding needed columns

# Remove useless column "Unnamed: 0"
data = data.drop("Unnamed: 0", axis=1)

data["genres"] = data["genres"].apply(lambda x: x.split(", "))

data.drop(data[data["duration"] < 0].index, inplace = True)

data["datetime"] = pd.to_datetime(data["datetime"], errors = 'coerce')
data["release_date"] = pd.to_datetime(data["release_date"], errors = 'coerce')

In [5]:
# Show clean_data df
data.head(10)

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","[Comedy, Drama, Romance]",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"[Fantasy, Horror, Mystery, Thriller]",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"[Action, Thriller]",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"[Action, Drama]",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"[Animation, Action, Adventure, Comedy, Family,...",2004-11-19,a80d6fc2aa,a57c992287
5,2017-01-01 19:21:37,0.0,London Has Fallen,"[Action, Thriller]",2016-03-04,f77e500e7a,c5bf4f3f57
6,2017-01-01 19:43:06,4903.0,The Water Diviner,"[Drama, History, War]",2014-12-26,7165c2fc94,8e1be40e32
7,2017-01-01 19:44:38,0.0,Angel of Christmas,"[Comedy, Romance]",2015-11-29,b2f02f2689,892a51dee1
8,2017-01-01 19:46:24,3845.0,Ratter,"[Drama, Horror, Thriller]",2016-02-12,c39aae36c3,cff8ea652a
9,2017-01-01 20:27:04,0.0,The Book of Life,"[Animation, Adventure, Comedy, Family, Fantasy...",2014-10-17,97183b9136,bf53608c70


Now we need to extract the requested features.
The functions used to extract each feature are contained in the "feature_functions.py" file.
This is not the most efficient way of obtaining this results (user_id groupby is repeated in each func),
but we decided to keep it for code clarity.

In [7]:
#Favorite genre (i.e., the genre on which the user spent the most time)
def favourite_genere(df):
    df.drop(df[df.duration == 0].index)
    
    # Explode the list of genres into separate rows
    df_expanded = df.explode('genres')

    # Group by 'user_id' and 'genre', summing up the 'time_spent'
    grouped_df = df_expanded.groupby(['user_id', 'genres'])['duration'].sum().reset_index()

    # Find the index of the maximum 'time_spent' for each user
    idx = grouped_df.groupby('user_id')['duration'].idxmax()

    # Create a DataFrame with only the rows corresponding to the maximum 'time_spent' for each user
    favorite_genre_df = grouped_df.loc[idx]

    # Rename columns for clarity
    favorite_genre_df = favorite_genre_df.rename(columns={'genres': 'favorite_genre', 'duration': 'time_spent_on_favorite_genre'})

    # Merge the new DataFrame back to the original DataFrame based on 'user_id'
    df = pd.merge(df, favorite_genre_df[['user_id', 'favorite_genre']], on='user_id', how='left')

    # Display the updated DataFrame
    return(df)

In [8]:
data = favourite_genere(data)

In [9]:
def avg_click_time(df):
    average_click_time_df = df.groupby('user_id')['duration'].mean().reset_index()

    # Rename the columns for clarity
    average_click_time_df = average_click_time_df.rename(columns={'duration': 'average_click_time'})

    # Merge the new DataFrame back to the original DataFrame based on 'user_id'
    df = pd.merge(df, average_click_time_df, on='user_id', how='left')

    # Display the updated DataFrame
    return(df)

In [10]:
data = avg_click_time(data)

In [11]:
def time_day(df):  
    bins = [0, 6, 13, 20, 24]
    labels = ['Night', 'Morning', 'Afternoon', 'Night']

    # Create a new column 'time_of_day'
    df['time_of_day'] = pd.cut(df['datetime'].dt.hour, bins=bins, labels=labels, include_lowest=True, ordered=False)

    time_of_day_summary = df.groupby('time_of_day')['duration'].sum().reset_index()

    # Find the time_of_day with the maximum total duration
    max_time_of_day = time_of_day_summary.loc[time_of_day_summary['duration'].idxmax(), 'time_of_day']

    # Add a new column 'max_time_of_day' to the original DataFrame
    df['max_time_of_day'] = max_time_of_day
    return(df)


In [12]:
data = time_day(data)

In [13]:
def old_movie(df):
    df['release_date'] = pd.to_datetime(df['release_date'])
    threshold_date = pd.Timestamp('2010-01-01')
    df['movie_preference'] = df['release_date'].apply(lambda date: 'Old Movie Lover' if date <= threshold_date else 'Recent Content Lover')
    return(df)

In [14]:
data = old_movie(data)

In [15]:
def avg_timespent(df):
    grouped_df = df.groupby(['user_id', 'datetime'])['duration'].sum().reset_index()
    average_duration_per_day = grouped_df.groupby('user_id')['duration'].mean().reset_index()
    average_duration_per_day = average_duration_per_day.rename(columns={'duration': 'average_duration_per_day'})
    df = pd.merge(df, average_duration_per_day, on='user_id', how='left')
    return(df)

In [16]:
data = avg_timespent(data)

In [17]:
data[data["user_id"]=="b15926c011"]

,datetime,duration,title,genres,release_date,movie_id,user_id,favorite_genre,average_click_time,time_of_day,max_time_of_day,movie_preference,average_duration_per_day
307113,2018-03-21 20:42:47,2400.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Afternoon,Afternoon,Old Movie Lover,3324.693351
307136,2018-03-21 21:22:47,179456.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Night,Afternoon,Old Movie Lover,3324.693351
307990,2018-03-23 23:15:47,1217.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Night,Afternoon,Old Movie Lover,3324.693351
308011,2018-03-23 23:36:04,1401.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Night,Afternoon,Old Movie Lover,3324.693351
308035,2018-03-23 23:59:25,2635.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Night,Afternoon,Old Movie Lover,3324.693351
...,...,...,...,...,...,...,...,...,...,...,...,...,...
348142,2018-05-14 00:04:10,1444.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Night,Afternoon,Old Movie Lover,3324.693351
348162,2018-05-14 00:28:14,546.0,Wild Child,"[Comedy, Drama, Romance]",2008-08-15,c424c83faf,b15926c011,Comedy,3324.693351,Night,Afternoon,Old Movie Lover,3324.693351
348480,2018-05-15 20:36:56,46.0,Coin Heist,"[Crime, Drama, Romance, Thriller]",2017-01-06,57e2731b38,b15926c011,Comedy,3324.693351,Afternoon,Afternoon,Recent Content Lover,3324.693351
348495,2018-05-15 20:37:42,86.0,Coin Heist,"[Crime, Drama, Romance, Thriller]",2017-01-06,57e2731b38,b15926c011,Comedy,3324.693351,Afternoon,Afternoon,Recent Content Lover,3324.693351


With the help of ChatGPT and our imagination we thouht of 10 other features we could add to the dataset
ChatGPT helped us select the following:
- average number of movies watched on weekdays vs. weekends
- average number of movies watched on holidays vs. non-holidays
- genres diversity
- number of unique shows watched
- zapping percentage
- average number of shows per month
- time since last watch
- ? amount of time spent when movie/serie is released 
- ? amount of times a serie/movie has been rewatched by the same user

In [18]:
#average number of movies watched on weekdays vs. weekends

def week_end(df):
    df['week_end'] = df['datetime'].dt.weekday.map({0: 'weekday', 1: 'weekday', 2: 'weekday', 3: 'weekday', 4: 'weekday', 5: 'weekend', 6: 'weekend'})
    avg_duration_weekorend = df.groupby('week_end')['duration'].mean().reset_index()
    avg_duration_weekorend = avg_duration_weekorend.rename(columns={'duration': 'avg_duration_weekorend'})
    df = pd.merge(df, avg_duration_weekorend, on='week_end', how='left')
    return(df)


In [19]:
data = week_end(data)

In [20]:
def genere_diversity(df):
    unique_genres = df.explode('genres').groupby('user_id')['genres'].nunique().reset_index()
    
    # Calculate a diversity metric (e.g., Shannon entropy) I asked how to do this to ChatGPT
    unique_genres['genre_diversity'] = -1 * unique_genres['genres'] * \
    unique_genres['genres'].apply(lambda x: x / unique_genres['genres'].sum()).apply(lambda x: 0 if pd.isna(x) else x * (np.log(x) if x != 0 else 0))
    unique_genres = unique_genres.rename(columns={'genres': 'unique_genres'})
    df = pd.merge(df, unique_genres[['user_id', 'unique_genres']], on='user_id', how='left')
    df = pd.merge(df, unique_genres[['user_id', 'genre_diversity']], on='user_id', how='left')
    return(df)

In [21]:
data = genere_diversity(data)

In [22]:
# number of unique shows watched

def unique_shows(df):
    unique_movies_count = df.groupby('user_id')['movie_id'].nunique().reset_index()
    # Rename the column for clarity
    unique_movies_count = unique_movies_count.rename(columns={'movie_id': 'unique_movies_count'})
    df = pd.merge(df, unique_movies_count, on='user_id', how='left')
    return(df)

In [23]:
data = unique_shows(data)

In [24]:
def zapping(df):
    df['time_diff'] = df.groupby('user_id')['datetime'].diff()

    time_threshold = pd.Timedelta('5 minutes')

    # Identify zapping events based on the time threshold
    df['zapping_event'] = (df['time_diff'] <= time_threshold) & (df['time_diff'].notna())

    # Calculate the zapping percentage for each user
    zapping_percentage = df.groupby('user_id')['zapping_event'].mean().reset_index()
    zapping_percentage = zapping_percentage.rename(columns={'zapping_event': 'zapping_percentage'})

    # Merge back to the original DataFrame based on 'user_id'
    df = pd.merge(df, zapping_percentage, on='user_id', how='left')
    columns_to_drop = ['zapping_event', 'time_diff']
    df = df.drop(columns=columns_to_drop)
    return(df)


In [25]:
data = zapping(data)

In [26]:
#average number of movies per month
def avg_movies_month(df):
    df['month'] = df['datetime'].dt.to_period('M')

    # Group by user_id and month, then count the number of shows
    average_shows_per_month = df.groupby(['user_id', 'month'])['movie_id'].count().reset_index()

    # Calculate the average number of shows per month per user
    average_shows_per_month = average_shows_per_month.groupby('user_id')['movie_id'].mean().reset_index()
    average_shows_per_month = average_shows_per_month.rename(columns={'movie_id': 'average_movies_per_month'})

    df = pd.merge(df, average_shows_per_month, on='user_id', how='left')
    columns_to_drop = ['month']
    df = df.drop(columns=columns_to_drop)
    return(df)

In [27]:
data = avg_movies_month(data)

In [28]:
#time since last watch 
def last_time(df):
    df = df.sort_values(by=['user_id', 'datetime'])

    # Calculate the time difference between consecutive rows for each user
    df['time_since_last_watch'] = df.groupby('user_id')['datetime'].diff()
    return(df)

In [29]:
data = last_time(data)

In [30]:
#amount of times a movie has been rewathced for each user
def rewatch(df):
    df = df.sort_values(by=['user_id', 'datetime'])

    # Calculate the time difference between consecutive rows for each user and movie/series
    df['time_diff'] = df.groupby(['user_id', 'movie_id'])['datetime'].diff()

    # Filter entries where the time difference is greater than a threshold (e.g., 1 day)
    rewatched_df = df[df['time_diff'] > pd.Timedelta('1 day')]

    # Count the number of times each movie/series has been rewatched by the same user
    rewatch_count = rewatched_df.groupby(['user_id', 'movie_id']).size().reset_index(name='rewatch_count')

    df = pd.merge(df, rewatch_count, on='user_id', how='left')
    return(df)


In [31]:
rewatch(data)

,datetime,duration,title,genres,release_date,movie_id_x,user_id,favorite_genre,average_click_time,time_of_day,...,avg_duration_weekorend,unique_genres,genre_diversity,unique_movies_count,zapping_percentage,average_movies_per_month,time_since_last_watch,time_diff,movie_id_y,rewatch_count
0,2017-12-05 20:39:15,0.0,Hannibal,"[Crime, Drama, Thriller]",2001-02-09,9bfee795ff,00004e2862,Crime,0.000000,Afternoon,...,35422.016886,3,0.000136,1,0.000000,1.0,NaT,NaT,NaN,NaN
1,2017-05-23 18:41:44,2957.0,Frailty,"[Crime, Drama, Thriller]",2002-04-12,7314699c23,000052a0a0,Action,2024.166667,Afternoon,...,35422.016886,14,0.002602,11,0.416667,12.0,NaT,NaT,NaN,NaN
2,2017-05-23 19:47:31,1512.0,Frailty,"[Crime, Drama, Thriller]",2002-04-12,7314699c23,000052a0a0,Action,2024.166667,Afternoon,...,35422.016886,14,0.002602,11,0.416667,12.0,0 days 01:05:47,0 days 01:05:47,NaN,NaN
3,2017-05-23 21:37:40,1198.0,Frailty,"[Crime, Drama, Thriller]",2002-04-12,7314699c23,000052a0a0,Action,2024.166667,Night,...,35422.016886,14,0.002602,11,0.416667,12.0,0 days 01:50:09,0 days 01:50:09,NaN,NaN
4,2017-05-24 07:29:07,0.0,Resident Evil,"[Action, Horror, Sci-Fi]",2002-03-15,6275614f9a,000052a0a0,Action,2024.166667,Morning,...,35422.016886,14,0.002602,11,0.416667,12.0,0 days 09:51:27,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014322,2017-09-18 21:03:08,173436.0,Fight Club,[Drama],1999-10-15,338abadc17,fffeac83be,Drama,40606.272727,Night,...,35422.016886,9,0.001119,10,0.090909,5.5,7 days 23:36:50,NaT,NaN,NaN
1014323,2017-09-20 21:18:18,129.0,To the Bone,[Drama],2017-07-14,5a650007e7,fffeac83be,Drama,40606.272727,Night,...,35422.016886,9,0.001119,10,0.090909,5.5,2 days 00:15:10,NaT,NaN,NaN
1014324,2017-07-23 14:45:12,0.0,Forks Over Knives,[Documentary],2013-10-16,9ab62a3f2c,ffff2c5f9e,Action,0.000000,Afternoon,...,33093.167544,5,0.000363,2,0.000000,2.0,NaT,NaT,NaN,NaN
1014325,2017-07-23 15:02:43,0.0,Hot Fuzz,"[Action, Comedy, Mystery, Thriller]",2007-04-20,6467fee6b6,ffff2c5f9e,Action,0.000000,Afternoon,...,33093.167544,5,0.000363,2,0.000000,2.0,0 days 00:17:31,NaT,NaN,NaN


In [32]:
print(data.dtypes)

datetime                     datetime64[ns]
duration                            float64
title                                object
genres                               object
release_date                 datetime64[ns]
movie_id                             object
user_id                              object
favorite_genre                       object
average_click_time                  float64
time_of_day                        category
max_time_of_day                      object
movie_preference                     object
average_duration_per_day            float64
week_end                             object
avg_duration_weekorend              float64
unique_genres                         int64
genre_diversity                     float64
unique_movies_count                   int64
zapping_percentage                  float64
average_movies_per_month            float64
time_since_last_watch       timedelta64[ns]
dtype: object
